In [1]:
import argparse
import os
import json
import sys
from codebase.dataset import Dataset
from codebase.models import *
from codebase.trainer import Trainer, make_dir_if_not_exist
from codebase.defaults import get_default_kwargs
import tensorflow as tf


2024-01-15 16:59:54.513673: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 16:59:54.594071: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-15 16:59:54.594121: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-15 16:59:54.595742: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-15 16:59:54.605480: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-15 16:59:54.607197: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
parser = argparse.ArgumentParser(description='Run model')
parser.add_argument('-d', '--dataset', help='dataset name', default='adult_sex')
parser.add_argument('-n', '--expname', help='experiment name', default='temp')
parser.add_argument('-pc', '--pass_coeff', help='coefficient on pass term', default=1., type=float)
parser.add_argument('-fc', '--fair_coeff', help='coefficient on fairness term', default=1., type=float)
parser.add_argument('-pass', '--pass_model', action='store_true', help='run PASS model?', default=False)
parser.add_argument('-def', '--defer', action='store_true', help='learn to defer? (as opposed to reject)', default=True)
parser.add_argument('-ne', '--num_epochs', help='number of training epochs', default=10000, type=int)
parser.add_argument('-pa', '--patience', help='training patience for early stopping', default=30, type=int)
parser.add_argument('-ua', '--use_attr', action='store_true', help='use sensitive attribute in data', default=True)
parser.add_argument('-mrs', '--model_random_seed', help='what random seed to use for model', default=0, type=int)
parser.add_argument('-drs', '--data_random_seed', help='what random seed to use for data', default=0, type=int)
parser.add_argument('-dm', '--dm_type', help='type of DM to use: inconsistent/biased/highacc', default='inconsistent')
parser.add_argument('-dirs', '--dirconf', help='config file for dirs', default='conf/dirs/home.json')
args = vars(parser.parse_args([]))


In [3]:
#load options
dirs = json.load(open(args['dirconf'], 'r'))
model_args = json.load(open('conf/model/{}.json'.format(args['dataset'])))

# get default params
data_kwargs, model_kwargs = get_default_kwargs(dirs['data'], args, model_args)

In [6]:
#change data kwargs
experiment_info = {}
experiment_info["K"] = []
experiment_info["budget"] = []
experiment_info["Error"] = []
experiment_info["Equal opportunity difference"] = []
experiment_info["fc"] = []
fcs = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 1,1.5,2,3,4,5]
for fc in fcs:
    model_kwargs["fair_coeff"] = fc
    for K in [1,2,3,4,5]:
        for budget in [1, 0.1, 0.01]:
            tf.compat.v1.reset_default_graph()
            data_kwargs['npzfile'] = "data/adult/adult{}_sex.npz".format(K,budget)
            #get dataset
            data = Dataset(**data_kwargs,budget=budget)
            model = MLPDeferModel(**model_kwargs)

            with tf.compat.v1.Session() as sess:
                print('Session created.')
                resdirname = os.path.join(dirs['exp'], args['dataset'])
                logdirname = os.path.join(dirs['log'], args['dataset'])
                ckptdirname = os.path.join(resdirname, 'checkpoints')
                for d in [resdirname, logdirname, ckptdirname]:
                    make_dir_if_not_exist(d)
                #create Trainer
                trainer = Trainer(model, data, sess=sess, logs_path=logdirname, \
                            checkpoint_path=ckptdirname, results_path=resdirname)

                save_path = trainer.train(n_epochs=args['num_epochs'], patience=args['patience'])
                trainer.restore(save_path)
                error,eod = trainer.test()
                print('Complete.')
                experiment_info["K"].append(K)
                experiment_info["budget"].append(budget)
                experiment_info["Error"].append(error)
                experiment_info["Equal opportunity difference"].append(eod)
                experiment_info["fc"].append(fc)


Session created.
Train: E0: loss:0.506, class_loss:0.440, fair_reg:0.180, idk_loss:0.066, loss_class:0.440, loss_idk:0.037, errRate:0.206, DI:0.113, errRate-T:0.203, DI-T:0.119, rejErrRate:0.204, IDKRate:0.034 | Valid: E0: loss:0.430, class_loss:0.425, fair_reg:0.186, idk_loss:0.005, loss_class:0.425, loss_idk:0.003, errRate:0.195, DI:0.117, errRate-T:0.195, DI-T:0.117, rejErrRate:0.195, IDKRate:0.000
Best validation loss so far of 0.430, model saved to /home/hc838/AuFair/predict-responsibly/experiments/adult_sex/checkpoints/model.ckpt
Train: E1: loss:0.436, class_loss:0.433, fair_reg:0.187, idk_loss:0.003, loss_class:0.433, loss_idk:0.002, errRate:0.198, DI:0.114, errRate-T:0.198, DI-T:0.114, rejErrRate:0.198, IDKRate:0.000 | Valid: E1: loss:0.424, class_loss:0.422, fair_reg:0.189, idk_loss:0.002, loss_class:0.422, loss_idk:0.001, errRate:0.193, DI:0.122, errRate-T:0.193, DI-T:0.122, rejErrRate:0.193, IDKRate:0.000
Train: E2: loss:0.433, class_loss:0.431, fair_reg:0.190, idk_loss:0.00

Train: E21: loss:0.419, class_loss:0.419, fair_reg:0.191, idk_loss:0.000, loss_class:0.419, loss_idk:0.000, errRate:0.190, DI:0.077, errRate-T:0.190, DI-T:0.077, rejErrRate:0.190, IDKRate:0.000 | Valid: E21: loss:0.422, class_loss:0.422, fair_reg:0.202, idk_loss:0.000, loss_class:0.422, loss_idk:0.000, errRate:0.192, DI:0.094, errRate-T:0.192, DI-T:0.094, rejErrRate:0.192, IDKRate:0.000
Train: E22: loss:0.419, class_loss:0.419, fair_reg:0.191, idk_loss:0.000, loss_class:0.419, loss_idk:0.000, errRate:0.190, DI:0.078, errRate-T:0.190, DI-T:0.078, rejErrRate:0.190, IDKRate:0.000 | Valid: E22: loss:0.422, class_loss:0.422, fair_reg:0.202, idk_loss:0.000, loss_class:0.422, loss_idk:0.000, errRate:0.192, DI:0.090, errRate-T:0.192, DI-T:0.090, rejErrRate:0.192, IDKRate:0.000
Train: E23: loss:0.419, class_loss:0.419, fair_reg:0.191, idk_loss:0.000, loss_class:0.419, loss_idk:0.000, errRate:0.190, DI:0.078, errRate-T:0.190, DI-T:0.078, rejErrRate:0.190, IDKRate:0.000 | Valid: E23: loss:0.422, 

Train: E3: loss:0.462, class_loss:0.438, fair_reg:0.174, idk_loss:0.024, loss_class:0.438, loss_idk:0.013, errRate:0.209, DI:0.121, errRate-T:0.209, DI-T:0.121, rejErrRate:0.209, IDKRate:0.000 | Valid: E3: loss:0.395, class_loss:0.375, fair_reg:0.203, idk_loss:0.019, loss_class:0.375, loss_idk:0.012, errRate:0.177, DI:0.149, errRate-T:0.177, DI-T:0.149, rejErrRate:0.177, IDKRate:0.000
Train: E4: loss:0.451, class_loss:0.435, fair_reg:0.177, idk_loss:0.016, loss_class:0.435, loss_idk:0.009, errRate:0.207, DI:0.122, errRate-T:0.207, DI-T:0.122, rejErrRate:0.207, IDKRate:0.000 | Valid: E4: loss:0.388, class_loss:0.375, fair_reg:0.209, idk_loss:0.014, loss_class:0.375, loss_idk:0.009, errRate:0.175, DI:0.154, errRate-T:0.175, DI-T:0.154, rejErrRate:0.175, IDKRate:0.000
Train: E5: loss:0.445, class_loss:0.433, fair_reg:0.180, idk_loss:0.012, loss_class:0.433, loss_idk:0.007, errRate:0.203, DI:0.117, errRate-T:0.203, DI-T:0.117, rejErrRate:0.203, IDKRate:0.000 | Valid: E5: loss:0.385, class_

Train: E24: loss:0.417, class_loss:0.416, fair_reg:0.189, idk_loss:0.001, loss_class:0.416, loss_idk:0.001, errRate:0.188, DI:0.093, errRate-T:0.188, DI-T:0.093, rejErrRate:0.188, IDKRate:0.000 | Valid: E24: loss:0.396, class_loss:0.395, fair_reg:0.201, idk_loss:0.001, loss_class:0.395, loss_idk:0.001, errRate:0.185, DI:0.158, errRate-T:0.185, DI-T:0.158, rejErrRate:0.185, IDKRate:0.000
Train: E25: loss:0.417, class_loss:0.415, fair_reg:0.189, idk_loss:0.001, loss_class:0.415, loss_idk:0.001, errRate:0.187, DI:0.082, errRate-T:0.187, DI-T:0.082, rejErrRate:0.187, IDKRate:0.000 | Valid: E25: loss:0.397, class_loss:0.396, fair_reg:0.200, idk_loss:0.001, loss_class:0.396, loss_idk:0.001, errRate:0.185, DI:0.158, errRate-T:0.185, DI-T:0.158, rejErrRate:0.185, IDKRate:0.000
Train: E26: loss:0.416, class_loss:0.414, fair_reg:0.189, idk_loss:0.001, loss_class:0.414, loss_idk:0.001, errRate:0.185, DI:0.084, errRate-T:0.185, DI-T:0.084, rejErrRate:0.185, IDKRate:0.000 | Valid: E26: loss:0.398, 

Best validation loss so far of 1.112, model saved to /home/hc838/AuFair/predict-responsibly/experiments/adult_sex/checkpoints/model.ckpt
Train: E1: loss:1.099, class_loss:0.439, fair_reg:0.046, idk_loss:0.660, loss_class:0.439, loss_idk:0.371, errRate:0.230, DI:0.000, errRate-T:0.157, DI-T:0.080, rejErrRate:nan, IDKRate:1.000 | Valid: E1: loss:1.019, class_loss:0.426, fair_reg:0.356, idk_loss:0.593, loss_class:0.426, loss_idk:0.341, errRate:0.167, DI:0.000, errRate-T:0.188, DI-T:0.393, rejErrRate:nan, IDKRate:1.000
Train: E2: loss:1.013, class_loss:0.444, fair_reg:0.059, idk_loss:0.569, loss_class:0.444, loss_idk:0.317, errRate:0.230, DI:0.000, errRate-T:0.157, DI-T:0.080, rejErrRate:nan, IDKRate:1.000 | Valid: E2: loss:0.939, class_loss:0.426, fair_reg:0.187, idk_loss:0.513, loss_class:0.426, loss_idk:0.295, errRate:0.167, DI:0.000, errRate-T:0.188, DI-T:0.393, rejErrRate:0.000, IDKRate:0.979
Train: E3: loss:0.939, class_loss:0.446, fair_reg:0.044, idk_loss:0.493, loss_class:0.446, lo

...

Train: E26: loss:0.580, class_loss:0.447, fair_reg:0.132, idk_loss:0.001, loss_class:0.447, loss_idk:0.001, errRate:0.221, DI:0.099, errRate-T:0.221, DI-T:0.099, rejErrRate:0.221, IDKRate:0.000 | Valid: E26: loss:0.626, class_loss:0.410, fair_reg:0.214, idk_loss:0.001, loss_class:0.410, loss_idk:0.001, errRate:0.181, DI:0.125, errRate-T:0.181, DI-T:0.125, rejErrRate:0.181, IDKRate:0.000
Train: E27: loss:0.576, class_loss:0.444, fair_reg:0.131, idk_loss:0.001, loss_class:0.444, loss_idk:0.001, errRate:0.219, DI:0.127, errRate-T:0.219, DI-T:0.127, rejErrRate:0.219, IDKRate:0.000 | Valid: E27: loss:0.613, class_loss:0.414, fair_reg:0.198, idk_loss:0.001, loss_class:0.414, loss_idk:0.001, errRate:0.183, DI:0.122, errRate-T:0.183, DI-T:0.122, rejErrRate:0.183, IDKRate:0.000
Train: E28: loss:0.578, class_loss:0.443, fair_reg:0.135, idk_loss:0.001, loss_class:0.443, loss_idk:0.001, errRate:0.219, DI:0.112, errRate-T:0.219, DI-T:0.112, rejErrRate:0.219, IDKRate:0.000 | Valid: E28: loss:0.620, 

Train: E4: loss:1.212, class_loss:0.461, fair_reg:0.312, idk_loss:0.440, loss_class:0.461, loss_idk:0.237, errRate:0.208, DI:0.000, errRate-T:0.208, DI-T:0.000, rejErrRate:0.208, IDKRate:0.000 | Valid: E4: loss:0.904, class_loss:0.431, fair_reg:0.072, idk_loss:0.401, loss_class:0.431, loss_idk:0.228, errRate:0.146, DI:0.000, errRate-T:0.146, DI-T:0.000, rejErrRate:0.146, IDKRate:0.000
Train: E5: loss:1.054, class_loss:0.464, fair_reg:0.206, idk_loss:0.384, loss_class:0.464, loss_idk:0.205, errRate:0.208, DI:0.000, errRate-T:0.208, DI-T:0.000, rejErrRate:0.208, IDKRate:0.000 | Valid: E5: loss:0.830, class_loss:0.426, fair_reg:0.054, idk_loss:0.349, loss_class:0.426, loss_idk:0.200, errRate:0.146, DI:0.000, errRate-T:0.146, DI-T:0.000, rejErrRate:0.146, IDKRate:0.000
Train: E6: loss:1.036, class_loss:0.463, fair_reg:0.240, idk_loss:0.334, loss_class:0.463, loss_idk:0.179, errRate:0.208, DI:0.000, errRate-T:0.208, DI-T:0.000, rejErrRate:0.208, IDKRate:0.000 | Valid: E6: loss:0.756, class_

Best validation loss so far of 0.549, model saved to /home/hc838/AuFair/predict-responsibly/experiments/adult_sex/checkpoints/model.ckpt
Train: E25: loss:0.588, class_loss:0.371, fair_reg:0.179, idk_loss:0.038, loss_class:0.371, loss_idk:0.024, errRate:0.161, DI:0.085, errRate-T:0.161, DI-T:0.085, rejErrRate:0.161, IDKRate:0.000 | Valid: E25: loss:0.571, class_loss:0.398, fair_reg:0.137, idk_loss:0.036, loss_class:0.398, loss_idk:0.022, errRate:0.146, DI:0.107, errRate-T:0.146, DI-T:0.107, rejErrRate:0.146, IDKRate:0.000
Train: E26: loss:0.579, class_loss:0.370, fair_reg:0.174, idk_loss:0.035, loss_class:0.370, loss_idk:0.022, errRate:0.161, DI:0.123, errRate-T:0.161, DI-T:0.123, rejErrRate:0.161, IDKRate:0.000 | Valid: E26: loss:0.578, class_loss:0.397, fair_reg:0.147, idk_loss:0.034, loss_class:0.397, loss_idk:0.021, errRate:0.146, DI:0.107, errRate-T:0.146, DI-T:0.107, rejErrRate:0.146, IDKRate:0.000
Train: E27: loss:0.572, class_loss:0.367, fair_reg:0.171, idk_loss:0.033, loss_clas

Train: E46: loss:0.452, class_loss:0.304, fair_reg:0.134, idk_loss:0.014, loss_class:0.304, loss_idk:0.010, errRate:0.146, DI:0.060, errRate-T:0.146, DI-T:0.060, rejErrRate:0.146, IDKRate:0.000 | Valid: E46: loss:0.617, class_loss:0.433, fair_reg:0.170, idk_loss:0.014, loss_class:0.433, loss_idk:0.008, errRate:0.167, DI:0.193, errRate-T:0.167, DI-T:0.193, rejErrRate:0.167, IDKRate:0.000
Train: E47: loss:0.448, class_loss:0.303, fair_reg:0.132, idk_loss:0.014, loss_class:0.303, loss_idk:0.009, errRate:0.156, DI:0.071, errRate-T:0.156, DI-T:0.071, rejErrRate:0.156, IDKRate:0.000 | Valid: E47: loss:0.618, class_loss:0.436, fair_reg:0.169, idk_loss:0.013, loss_class:0.436, loss_idk:0.008, errRate:0.167, DI:0.193, errRate-T:0.167, DI-T:0.193, rejErrRate:0.167, IDKRate:0.000
Train: E48: loss:0.432, class_loss:0.302, fair_reg:0.117, idk_loss:0.013, loss_class:0.302, loss_idk:0.009, errRate:0.151, DI:0.228, errRate-T:0.151, DI-T:0.228, rejErrRate:0.151, IDKRate:0.000 | Valid: E48: loss:0.620, 

Train: E69: loss:0.381, class_loss:0.265, fair_reg:0.109, idk_loss:0.007, loss_class:0.265, loss_idk:0.005, errRate:0.120, DI:0.171, errRate-T:0.120, DI-T:0.171, rejErrRate:0.120, IDKRate:0.000 | Valid: E69: loss:0.677, class_loss:0.491, fair_reg:0.179, idk_loss:0.007, loss_class:0.491, loss_idk:0.004, errRate:0.188, DI:0.218, errRate-T:0.188, DI-T:0.218, rejErrRate:0.188, IDKRate:0.000
Train: E70: loss:0.376, class_loss:0.261, fair_reg:0.109, idk_loss:0.007, loss_class:0.261, loss_idk:0.005, errRate:0.115, DI:0.166, errRate-T:0.115, DI-T:0.166, rejErrRate:0.115, IDKRate:0.000 | Valid: E70: loss:0.672, class_loss:0.492, fair_reg:0.173, idk_loss:0.007, loss_class:0.492, loss_idk:0.003, errRate:0.188, DI:0.218, errRate-T:0.188, DI-T:0.218, rejErrRate:0.188, IDKRate:0.000
Train: E71: loss:0.386, class_loss:0.259, fair_reg:0.120, idk_loss:0.007, loss_class:0.259, loss_idk:0.005, errRate:0.120, DI:0.089, errRate-T:0.120, DI-T:0.089, rejErrRate:0.120, IDKRate:0.000 | Valid: E71: loss:0.673, 

In [8]:
import pandas as pd
df = pd.DataFrame.from_dict(experiment_info)
#save df to csv
df.to_csv("results/adult_sex.csv",index=False)